In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
sumstat_dir = '/home/lucytian/data/4_polypred/gwas_geno'

lym = f"{sumstat_dir}/blood_count/pop_10PCs/WB_train_val/INI30120.glm.tsv.gz" #338650
ldl = f"{sumstat_dir}/blood_biochemistry/pop_10PCs/WB_train_val/INI20030780.glm.tsv.gz" #
fev = f"{sumstat_dir}/spirometry/pop_10PCs/WB_train_val/INI1003063.glm.tsv.gz"
egfr = f"{sumstat_dir}/blood_biochemistry/pop_10PCs/WB_train_val/INI50030700.glm.tsv.gz"

In [4]:
#APOE = pd.read_csv('/home/lucytian/group/data/geno/APOE.pvar', sep='\t')

In [3]:
snpvar = pd.read_csv('priors/snps_with_var.gz', sep='\t', compression='gzip')

In [6]:
dfs = []
#min_maf = 0.001
for p in [lym, ldl, fev, egfr]:
    match = re.search(r'INI\d+', p)
    if match:
        trait_id = match.group()
        print(trait_id)
    else:
        raise ValueError(f"Could not extract trait ID from path: {p}")
    
    df = pd.read_csv(p, compression='gzip', sep='\t')
    df = df.rename(columns={'#CHROM': 'CHR', 'POS': 'BP', 'REF': 'A2'})
    df = df[~df['CHR'].isin(['X', 'Y', 'MT'])]
    df['CHR'] = df['CHR'].astype(int)
    
    merged = df.merge(snpvar, on=['CHR', 'BP'], suffixes=("_sum", "_snpvar"))

    # Alleles match
    same = (merged['A1_sum'] == merged['A1_snpvar']) & (merged['A2_sum'] == merged['A2_snpvar'])

    # Alleles are flipped
    flipped = (merged['A1_sum'] == merged['A2_snpvar']) & (merged['A2_sum'] == merged['A1_snpvar'])

    # Keep only SNPs that are either matching or flipped
    mask = same | flipped
    merged = merged.loc[mask].copy()     # filter AND copy to avoid chained assignment
    flipped = flipped.loc[mask]          # resize flipped to match new merged
    

    # Flip beta where alleles are flipped
    merged.loc[flipped, 'BETA'] *= -1
    merged.loc[flipped, 'ALT_FREQS'] = 1 - merged.loc[flipped, 'ALT_FREQS']
    merged['effect_allele'] = np.where(flipped, merged['A2_sum'], merged['A1_sum'])
    merged['alt_allele']    = np.where(flipped, merged['A1_sum'], merged['A2_sum'])
    
    vc = merged['SNP'].value_counts()
    merged = merged[merged["SNP"].isin(vc[vc==1].index)]
    
    merged['Z'] = merged['BETA'] / merged['SE']
    #merged['N'] = 338650
    merged = merged[['SNP', 'CHR', 'BP', 'effect_allele', 'alt_allele', 'Z', 'SNPVAR', 'ALT_FREQS']].rename(columns={'effect_allele': 'A1', 'alt_allele': 'A2', 'ALT_FREQS': 'MAF'})
    merged.to_csv(f'sumstat/{trait_id}_sumstats.txt.gz', sep='\t', index=False, compression='gzip')
    #for chr_num in np.arange(1, 23):
        #print(len(merged[merged['CHR'] == chr_num]))
        #merged[merged['CHR'] == chr_num].to_csv(f'sumstat/{trait_id}/chr{chr_num}.finemap_sumstats.txt.gz', sep='\t', index=False, compression='gzip')

INI30120


/tmp/ipykernel_246870/729022475.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


INI20030780


/tmp/ipykernel_246870/729022475.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


INI1003063


/tmp/ipykernel_246870/729022475.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


INI50030700


/tmp/ipykernel_246870/729022475.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


In [6]:
# ls chr*.gz | sed -E 's/chr([0-9XYMT]+)_([0-9]+)_([0-9]+)\.gz/\1\t\2\t\3/' > ld_blocks.txt

In [4]:
p = lym

df = pd.read_csv(p, compression='gzip', sep='\t')
df = df.rename(columns={'#CHROM': 'CHR', 'POS': 'BP', 'REF': 'A2'})
df = df[~df['CHR'].isin(['X', 'Y', 'MT'])]
df['CHR'] = df['CHR'].astype(int)
    
merged = df.merge(snpvar, on=['CHR', 'BP'], suffixes=("_sum", "_snpvar"))

# Alleles match
same = (merged['A1_sum'] == merged['A1_snpvar']) & (merged['A2_sum'] == merged['A2_snpvar'])

# Alleles are flipped
flipped = (merged['A1_sum'] == merged['A2_snpvar']) & (merged['A2_sum'] == merged['A1_snpvar'])

mask = same | flipped
merged = merged.loc[mask].copy()     # filter AND copy to avoid chained assignment
flipped = flipped.loc[mask] 

# Flip beta where alleles are flipped
merged.loc[flipped, 'BETA'] *= -1
merged.loc[flipped, 'ALT_FREQS'] = 1 - merged.loc[flipped, 'ALT_FREQS']
merged['effect_allele'] = np.where(flipped, merged['A2_sum'], merged['A1_sum'])
merged['alt_allele']    = np.where(flipped, merged['A1_sum'], merged['A2_sum'])

merged[['ID_sum']].to_csv('snp_extract.txt', sep='\t', header=None, index=False)

vc = merged['SNP'].value_counts()
merged = merged[merged["SNP"].isin(vc[vc==1].index)]

idmap = merged[['ID_sum','SNP']].dropna().drop_duplicates()
idmap.to_csv('update_name.txt', sep='\t', header=False, index=False)

a1 = merged[['SNP', 'effect_allele']].drop_duplicates()
a1.to_csv('a1_effect.txt', sep='\t', header=False, index=False)

/tmp/ipykernel_233411/2166493117.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


In [5]:
merged

,CHR,BP,ID_sum,A2_sum,ALT,A1_sum,TEST,OBS_CT,BETA,SE,...,LOG10_P,ERRCODE,ALT_FREQS,SNP,A1_snpvar,A2_snpvar,ID_snpvar,SNPVAR,effect_allele,alt_allele
0,1,756604,1:756604:A:G,A,G,G,ADD,262234,0.003658,0.004088,...,0.430817,.,0.129674,rs3131962,A,G,1:756604:G:A,4.073300e-09,A,G
1,1,767096,1:767096:A:G,A,G,G,ADD,262258,0.125438,0.064850,...,1.275080,.,0.999556,rs115991721,A,G,1:767096:G:A,4.073300e-09,A,G
2,1,768448,1:768448:G:A,G,A,A,ADD,262260,-0.001802,0.004465,...,0.163338,.,0.894697,rs12562034,G,A,1:768448:A:G,4.073300e-09,G,A
3,1,779322,1:779322:A:G,A,G,G,ADD,262032,-0.003700,0.004119,...,0.432955,.,0.872527,rs4040617,A,G,1:779322:G:A,4.073300e-09,A,G
4,1,785989,1:785989:T:C,T,C,C,ADD,261576,0.003064,0.004098,...,0.342273,.,0.129010,rs2980300,T,C,1:785989:C:T,4.073300e-09,T,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237972,22,51174939,22:51174939:C:T,C,T,T,ADD,262378,-0.007275,0.006086,...,0.634701,.,0.946208,rs73174435,C,T,22:51174939:T:C,4.073300e-09,C,T
1237973,22,51175626,22:51175626:A:G,A,G,G,ADD,261135,0.007788,0.005741,...,0.757265,.,0.939165,rs3810648,A,G,22:51175626:G:A,4.073300e-09,A,G
1237974,22,51185848,22:51185848:G:A,G,A,A,ADD,262032,-0.012684,0.006253,...,1.371270,.,0.050570,rs3865764,G,A,22:51185848:A:G,4.073300e-09,G,A
1237975,22,51193629,22:51193629:A:G,A,G,G,ADD,262347,0.001290,0.005188,...,0.094935,.,0.924367,rs142680588,A,G,22:51193629:G:A,1.320200e-07,A,G


In [7]:
pd.read_csv('sumstat/INI30120_Afr_sumstats.txt.gz', delim_whitespace=True)

,SNP,CHR,BP,A1,A2,BETA,SE,SNPVAR,MAF
0,rs3131962,1,756604,A,G,-0.029974,0.021137,4.073300e-09,0.595795
1,rs115991721,1,767096,A,G,-0.057822,0.037398,4.073300e-09,0.906250
2,rs12562034,1,768448,G,A,-0.018932,0.039288,4.073300e-09,0.920074
3,rs4040617,1,779322,A,G,0.025500,0.021080,4.073300e-09,0.505431
4,rs2980300,1,785989,T,C,-0.051791,0.029199,4.073300e-09,0.800734
...,...,...,...,...,...,...,...,...,...
1232017,rs73174435,22,51174939,C,T,-0.089592,0.186787,4.073300e-09,0.996695
1232018,rs3810648,22,51175626,A,G,0.004385,0.024072,4.073300e-09,0.734518
1232019,rs3865764,22,51185848,G,A,-0.067018,0.105768,4.073300e-09,0.010117
1232020,rs142680588,22,51193629,A,G,0.014498,0.047984,1.320200e-07,0.948845


In [8]:
pd.read_csv('sumstat/INI30120_sumstats.txt.gz', delim_whitespace=True)

,SNP,CHR,BP,A1,A2,Z,SNPVAR,MAF
0,rs3131962,1,756604,A,G,0.894908,4.073300e-09,0.129674
1,rs115991721,1,767096,A,G,1.934285,4.073300e-09,0.999556
2,rs12562034,1,768448,G,A,-0.403563,4.073300e-09,0.894697
3,rs4040617,1,779322,A,G,-0.898320,4.073300e-09,0.872527
4,rs2980300,1,785989,T,C,0.747601,4.073300e-09,0.129010
...,...,...,...,...,...,...,...,...
1232017,rs73174435,22,51174939,C,T,-1.195485,4.073300e-09,0.946208
1232018,rs3810648,22,51175626,A,G,1.356699,4.073300e-09,0.939165
1232019,rs3865764,22,51185848,G,A,-2.028273,4.073300e-09,0.050570
1232020,rs142680588,22,51193629,A,G,0.248630,1.320200e-07,0.924367
